# TD2 part 2: Named entity recognition

Dans ce TD, nous allons prendre un datasets où les noms de personnes sont taggés.<br>
Nous allons transformer ces données en tenseurs X, y et attention_mask.<br>
Nous allons créer un modèle RNN pour prédire si un mot est un nom de personne.<br>
Nous allons ensuite créer la loop avec l'optimizer pour apprendre le modèle.<br>
Du modèle appris (prédisant sur les tokens), nous allons postprocess les prédictions pour avoir les prédictions sur les noms.

Un fois que la loop est créée et que le modèle apprend, nous allons changer la structure du modèle:
- Changer learning rate. Comment se comporte le modèle
- Ajouter des couches denses, ReLU, dropout, normalization
- Changer le nombre de layers du RNN, LSTM.

Lorsqu'on a un bon modèle de prédiction pour les noms de personnes, nous allons l'appliquer à notre projet fil rouge.
Utilisez-le tel que. Quelle accuracy ?
Ré-entrainez la (les) dernière(s) couche(s) du modèle sur notre jeu de données. A-t-il gagné en accuracy ?

In [2]:
# Import
import matplotlib.pyplot as plt
import numpy as np
import torch
import transformers

/home/divino/miniconda3/envs/school-nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

Télécharger le dataset MultiNERD FR [ici](https://github.com/Babelscape/multinerd)<br>
Mettez les données dans le dossier data/raw du projet.


In [3]:
def extract_multinerd_person_words(filename="../data/raw/train_fr.tsv"):
    with open(filename) as f:
        tagged_words = [line.strip().split("\t") for line in f]
        
        # Joining words until we meet a dot
        # Word's label is 1 if 'PER' is in its tag
        sentences = []
        sentence_labels = []
    
        this_word = []
        this_labels = []
        for tagged_word in tagged_words:
            if len(tagged_word) < 3:
                # not a tagged word
                continue
            word = tagged_word[1]
            tag = tagged_word[2]
        
            if word == '.':
                sentences.append(" ".join(this_word))
                sentence_labels.append(this_labels)
            
                this_word = []
                this_labels = []
            else:
                this_word.append(word)
                this_labels.append(1 * tag.endswith("PER"))

    return sentences, sentence_labels


In [4]:
sentences, labels = extract_multinerd_person_words("../data/raw/multinerd_train.tsv")

## Tokenizer

En utilisant le tokenizer d'HuggingFace "camembert-base":
- Transformer les phrases en tokens
- Obtenez des vecteur y qui ont le même nombre d'entrées qu'il y a de tokens dans la phrase
- Ayez un tenseur "attention_mask" pour savoir sur quels tokens on cherche à predire le label
- Transformez les tokens en token_ids (avec le tokenizer)
Avec tout cela, vous pouvez former vos tenseurs X, Y et attention_mask

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("camembert-base")

In [6]:
def build_tokens_and_labels_and_attention_mask(tokenizer, sentence, labels):
    words = sentence.split()
    tokens = []
    tokens_label = []
    attention_mask = []
    
    for word, label in zip(words, labels):
        this_tokens = tokenizer.tokenize(word)
        tokens += this_tokens
        
        this_labels = [0] * len(this_tokens)
        this_labels[0] = label        
        tokens_label += this_labels
        
        this_attention_mask = [1] + [0] * (len(this_tokens) - 1)
        attention_mask += this_attention_mask
        
    return tokens, tokens_label, attention_mask

In [7]:
tokens, label, padding_masks = build_tokens_and_labels_and_attention_mask(tokenizer, sentences[0], labels[0])

## Model

Contruisez un modèle RNN comme dans la partie 1. Pour l'instant, il prendra comme arguments:
- Vocab size: le nombre de différents tokens du tokenizer (52 000 pour camembert-base)
- Embedding dim: la dimension de l'embedding des tokens (par défaut 50)
- hidden_dim: la dimension de l'état récurrent de votre RNN (par défaut 20)
- tagset_size: la nombre de classes possibles pour les prédictions (ici 2)

Dans le forward, votre modèle enchaînera les couches suivantes:
- Un embedding
- Un RNN
- Un ReLU
- Une couche linéaire
- Un softmax pour que la somme des prédictions pour une entrée soit égale à 1 (la prédiction pour un élément et sa probabilité d'être dans chaque classe)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

In [9]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=50, hidden_dim=20, tagset_size=2):
        super(RNNModel, self).__init__()
        
        # Initialisation des couches
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_dim, tagset_size)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, x):
        # Passage dans le modèle
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = self.relu(x)
        x = self.fc(x)
        x = self.softmax(x)
        return x


## Optimizer

Je fournis ici une fonction prenant un modèle, des tenseurs X, y, attention_mask.
Pour chaque batch:
- La loop utilise le modèle pour prédire sur x_batch
- Avec attention_mask, elle identifie sur quels tokens les prédictions compte
- Elle regarde la cross entropy entre y\[attention_ids\] et yhat\[attention_ids\]
- Elle output un dictionnaire avec le model et la loss au fur et à mesure des itérations

Entraînez le modèle avec vos données. <br>
Plottez la loss history.<br>
Itérez sur le modèle pour l'améliorer:
- Changer learning rate. Comment se comporte le modèle
- Ajouter des couches denses, ReLU, dropout, normalization
- Changer le nombre de layers du RNN, LSTM.



In [10]:
import gc

def train_model(model, X, y, attention_masks, n_epochs=100, lr=0.05, batch_size=128):
    loss_function = torch.nn.CrossEntropyLoss()
    loss_history = []

    dataset = torch.utils.data.TensorDataset(X, y, attention_masks)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        print("epoch:", epoch)
        for i, (x_batch, y_batch, mask) in enumerate(loader):
            optimizer.zero_grad()

            ids = mask.reshape(-1)
            yhat = model(x_batch).reshape((-1, 2))[ids]
            this_y = y_batch.reshape(-1)[ids]

            loss = loss_function(yhat, this_y)
            loss.backward()

            loss_history.append(loss.clone().detach())

            optimizer.step()

            gc.collect()
            torch.cuda.empty_cache()

        if epoch % 10 == 0:
            print(f"Got loss at {epoch}", np.mean(loss_history[-10:]))

    return {"model": model, "loss_history": loss_history}


In [11]:
vocab_size = 52000  # Pour camembert-base
model = RNNModel(vocab_size)

In [12]:
# Fonction pour construire les tenseurs à partir des données
def build_tensors(tokenizer, sentences, labels):
    X = []
    y = []
    attention_masks = []

    for sentence, label in zip(sentences, labels):
        tokens, tokens_label, attention_mask = build_tokens_and_labels_and_attention_mask(tokenizer, sentence, label)

        # Convertissez les tokens en ids
        token_ids = tokenizer.convert_tokens_to_ids(tokens)

        X.append(token_ids)
        y.append(tokens_label)
        attention_masks.append(attention_mask)

    # Padding pour garantir que toutes les séquences aient la même longueur
    X = torch.tensor([np.pad(token_ids, (0, max_length - len(token_ids)), mode='constant') for token_ids in X])
    y = torch.tensor([np.pad(label, (0, max_length - len(label)), mode='constant') for label in y])
    attention_masks = torch.tensor([np.pad(mask, (0, max_length - len(mask)), mode='constant') for mask in attention_masks])

    return X, y, attention_masks

# Création des tenseurs
max_length = max([len(tokenizer.encode(sentence)) for sentence in sentences])  # Longueur maximale des séquences
X, y, attention_masks = build_tensors(tokenizer, sentences, labels)

/tmp/ipykernel_8630/409453289.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X = torch.tensor([np.pad(token_ids, (0, max_length - len(token_ids)), mode='constant') for token_ids in X])


In [13]:
X = torch.tensor([np.pad(token_ids, (0, max_length - len(token_ids)), mode='constant') for token_ids in X], dtype=torch.long)
y = torch.tensor([np.pad(label, (0, max_length - len(label)), mode='constant') for label in y], dtype=torch.long)
attention_masks = torch.tensor([np.pad(mask, (0, max_length - len(mask)), mode='constant') for mask in attention_masks], dtype=torch.long)

In [14]:
results = train_model(model, X, y, attention_masks, n_epochs=2, batch_size=1)

epoch: 0


In [ ]:
import matplotlib.pyplot as plt

plt.plot(results['loss_history'])
plt.title('Loss History')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.show()

## Postprocessing

Créer une fonction prenant les prédictions du modèle (au niveau token) et sort les prédictions au niveau mot.<br>
Par exemple, admettons que, pour un mot, la prédiction du 1er token est la seule qu'on considère.<br>
si la phrase est "Bonjour John", avec les tokens \["bon", "jour", "Jo", "hn"\] avec les predictions \[0.12, 0.65, 0.88, 0.45\]<br>
Je veux récupérer les prédictions "bonjour": 0.12, "John": 0.88

In [ ]:
def postprocess_predictions(tokens, predictions, attention_masks):
    word_predictions = {}
    current_word = ""
    current_prediction = 0.0

    for token, prediction, mask in zip(tokens, predictions, attention_masks):
        if mask == 1:  # Début d'un nouveau mot
            if current_word:  # Si un mot précédent existe, l'ajouter au dictionnaire
                word_predictions[current_word] = current_prediction
            
            current_word = token  # Commencer un nouveau mot
            current_prediction = prediction
        else:
            current_word += token  # Continuer à construire le mot actuel

    # Ajouter le dernier mot s'il existe
    if current_word:
        word_predictions[current_word] = current_prediction

    return word_predictions


In [ ]:
for sentence in sentences:
    # Tokenisation
    tokens, tokens_label, attention_mask = build_tokens_and_labels_and_attention_mask(tokenizer, sentence, labels)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids_tensor = torch.tensor([token_ids])

    # Obtenir les prédictions du modèle
    with torch.no_grad():
        model_output = model(token_ids_tensor)
    predictions = torch.softmax(model_output, dim=2)
    predictions = predictions[0, :, 1]

    # Post-traitement pour obtenir les prédictions au niveau des mots
    word_predictions = postprocess_predictions(tokens, predictions.tolist(), attention_mask)

    print(f"Prédictions pour '{sentence}': {word_predictions}")
